In [2]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from pathlib import Path

In [6]:
score_type = 'mia'
df = pd.read_csv(f'/home/mathepa1/vscode-projects/mirror-mirror/output/mnist_bad_teacher_test/{score_type}_results.csv')
# df = pd.read_csv(f'/home/mathepa1/vscode-projects/mirror-mirror/output/cifar10_2024_08_07/{score_type}_results.csv')
score_label = f'{score_type.upper()} Score'
df.head()

,forget_set,run_number,model,score
0,0,0,control,0.098430
1,0,0,bad_teacher,0.000000
2,1,0,control,0.138090
3,1,0,bad_teacher,0.000000
4,2,0,control,0.051024


In [11]:
mean_df, std_df = df.groupby(['model', 'forget_set']).mean().drop('run_number', axis=1).reset_index(), df.groupby(['model', 'forget_set']).std().drop('run_number', axis=1).reset_index()
mean_df, std_df = mean_df.rename({'score': 'mean_score'}, axis=1), std_df.rename({'score': 'std_score'}, axis=1)
plot_df = pd.concat([mean_df.set_index(['model', 'forget_set']), std_df.set_index(['model', 'forget_set'])], axis=1).reset_index()
plot_df = plot_df[plot_df['forget_set'] != 'random_7000']
plot_df['forget_set'] = plot_df['forget_set'].apply(lambda x: x.split('_')[-1] if isinstance(x, str) else str(x))
plot_df = plot_df[plot_df['model'] != 'fisher_forgetting']
plot_df
plot_df.head()

,model,forget_set,mean_score,std_score
0,bad_teacher,0,0.000000,NaN
1,bad_teacher,1,0.000000,NaN
2,bad_teacher,2,0.000000,NaN
3,bad_teacher,3,0.000163,NaN
4,bad_teacher,4,0.000171,NaN


In [12]:
fig = px.bar(
    plot_df,
    x='forget_set',
    y='mean_score',
    color='model',
    labels={
        'forget_set': 'Forget Set',
        'mean_score': score_label,
        'model': 'Model',
    },
    barmode='group',
    error_y='std_score',
)
fig.update_xaxes(categoryorder='array', categoryarray=sorted(plot_df['forget_set'].unique(), key=(lambda x: int(x))))
fig.show()